In [1]:
import os
import sys

import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import split,decode,substring
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.ml import PipelineModel

from itertools import chain

import time

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

sc = SparkContext()
spark = SparkSession(sc)

localhost = '10.128.0.30'
int_ip = '10.128.0.30:9092'
topic = 'lab7topic'
Iris_path = "gs://me18b183_bdl/lab7kafka/Iris.csv"

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d50a887-c930-480a-a348-79b66c787fd7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.1.2/spark-sql-kafka-0-10_2.12-3.1.2.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2!spark-sql-

In [3]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", int_ip) \
    .option("subscribe", topic) \
    .load()

schema = StructType() \
    .add("SepalLength", FloatType()) \
    .add("SepalWidth", FloatType()) \
    .add("PetalLength", FloatType()) \
    .add("PetalWidth", FloatType()) \
    .add("Species", StringType())

In [4]:
df = df.select(f.from_json(f.decode(df.value, 'utf-8'), schema=schema).alias("input"))
df = df.select("input.*")

In [5]:
model = PipelineModel.load("gs://me18b183_bdl/lab7kafka/lab5/model")

df = df.withColumn('Actual_Species', df['Species'])

22/04/05 11:59:45 WARN org.apache.hadoop.util.concurrent.ExecutorHelper: Thread (Thread[GetFileInfo #1,5,main]) interrupted: 
java.lang.InterruptedException
	at com.google.common.util.concurrent.AbstractFuture.get(AbstractFuture.java:510)
	at com.google.common.util.concurrent.FluentFuture$TrustedFuture.get(FluentFuture.java:88)
	at org.apache.hadoop.util.concurrent.ExecutorHelper.logThrowableFromAfterExecute(ExecutorHelper.java:48)
	at org.apache.hadoop.util.concurrent.HadoopThreadPoolExecutor.afterExecute(HadoopThreadPoolExecutor.java:90)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1157)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)


In [6]:
pred = model.transform(df)

In [7]:
Species_map = dict(zip([0.0,1.0,2.0],[ 'Iris-setosa' ,'Iris-versicolor', 'Iris-virginica']))

mapping = f.create_map([f.lit(x) for x in chain(*Species_map.items())])
df_ = pred.withColumn('Predicted_Species', mapping[f.col("Prediction")])[['Actual_Species','Prediction','Predicted_Species']]

In [8]:
df_ = df_.withColumn('Accuracy' , f.when(f.col('Predicted_Species')==f.col('Actual_Species'),100).otherwise(0))

df_new = df_[['Actual_Species', 'Predicted_Species', 'Accuracy']]
df_new.createOrReplaceTempView('output')

In [ ]:
query = df_new.writeStream.queryName("output").outputMode('append').format('console').start()
query.awaitTermination()

22/04/05 12:00:00 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fcfcf8e5-1e40-4133-b714-71b1c292ee14. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/04/05 12:00:00 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
+--------------+-----------------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|   Iris-setosa|      Iris-setosa|     100|
+--------------+-----------------+--------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|   Iris-setosa|      Iris-setosa|     100|
|   Iris-setosa|      Iris-setosa|     100|
|   Iris-setosa|      Iris-setosa|     100|
+--------------+-----------------+--------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|   Iris-setosa|      Iris-setosa|     100|
+--------------+-----------------+--------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|   Iris-setosa|      Iris-setosa|     100|
+--------------+-----------------+--------+



...

-------------------------------------------
Batch: 114
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+



-------------------------------------------
Batch: 115
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+

-------------------------------------------
Batch: 116
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+

-------------------------------------------
Batch: 117
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+

----------------------------------------

22/04/05 12:03:05 WARN org.apache.hadoop.hdfs.DataStreamer: DataStreamer Exception
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.write(SocketChannelImpl.java:477)
	at org.apache.hadoop.net.SocketOutputStream$Writer.performIO(SocketOutputStream.java:63)
	at org.apache.hadoop.net.SocketIOWithTimeout.doIO(SocketIOWithTimeout.java:142)
	at org.apache.hadoop.net.SocketOutputStream.write(SocketOutputStream.java:159)
	at org.apache.hadoop.net.SocketOutputStream.write(SocketOutputStream.java:117)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:140)
	at java.io.DataOutputStream.flush(DataOutputStream.java:123)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:774)


-------------------------------------------
Batch: 130
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+



-------------------------------------------
Batch: 131
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+

-------------------------------------------
Batch: 132
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+

-------------------------------------------
Batch: 133
-------------------------------------------
+--------------+-----------------+--------+
|Actual_Species|Predicted_Species|Accuracy|
+--------------+-----------------+--------+
|Iris-virginica|   Iris-virginica|     100|
+--------------+-----------------+--------+

----------------------------------------